In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1054)
           2	LOAD_FAST(arg=0, lineno=1054)
           4	LOAD_CONST(arg=1, lineno=1054)
           6	BINARY_SUBSCR(arg=None, lineno=1054)
           8	LOAD_FAST(arg=0, lineno=1054)
          10	LOAD_CONST(arg=2, lineno=1054)
          12	BINARY_SUBSCR(arg=None, lineno=1054)
          14	COMPARE_OP(arg=4, lineno=1054)
          16	LOAD_FAST(arg=0, lineno=1054)
          18	LOAD_CONST(arg=1, lineno=1054)
          20	BINARY_SUBSCR(arg=None, lineno=1054)
          22	LOAD_FAST(arg=0, lineno=1054)
          24	LOAD_CONST(arg=3, lineno=1054)
          26	BINARY_SUBSCR(arg=None, lineno=1054)
          28	COMPARE_OP(arg=5, lineno=1054)
          30	BINARY_AND(arg=None, lineno=1054)
          32	RETURN_VALUE(arg=None, lineno=1054)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=105

In [2]:
hps = utils.get_hparams_from_file(r'./dlmodel/config.json')

## LJ Speech

In [3]:
len(symbols)

52

In [4]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
utils.load_checkpoint("dlmodel/chisato.pth", net_g, None)
_ = net_g.eval().to(torch.device('cpu'))

INFO:root:dec.cond.weight is not in the checkpoint
INFO:root:dec.cond.bias is not in the checkpoint
INFO:root:enc_q.enc.cond_layer.bias is not in the checkpoint
INFO:root:enc_q.enc.cond_layer.weight_g is not in the checkpoint
INFO:root:enc_q.enc.cond_layer.weight_v is not in the checkpoint
INFO:root:flow.flows.0.enc.cond_layer.bias is not in the checkpoint
INFO:root:flow.flows.0.enc.cond_layer.weight_g is not in the checkpoint
INFO:root:flow.flows.0.enc.cond_layer.weight_v is not in the checkpoint
INFO:root:flow.flows.2.enc.cond_layer.bias is not in the checkpoint
INFO:root:flow.flows.2.enc.cond_layer.weight_g is not in the checkpoint
INFO:root:flow.flows.2.enc.cond_layer.weight_v is not in the checkpoint
INFO:root:flow.flows.4.enc.cond_layer.bias is not in the checkpoint
INFO:root:flow.flows.4.enc.cond_layer.weight_g is not in the checkpoint
INFO:root:flow.flows.4.enc.cond_layer.weight_v is not in the checkpoint
INFO:root:flow.flows.6.enc.cond_layer.bias is not in the checkpoint
INFO:

In [7]:
# noise_scale：【0.1，1】控制感情变化程度
# noise_scale_w：【0.1，1】控制音素发音长度
# length_scale：【0.1，2】控制整体语速

# [ZH]你好，我叫千束[ZH]
# [JA]こんにちは、私はせ[JA]chisato
# Hello, My name is chisato.
stn_tst = get_text("[JA]こんにちは、私はせ[JA]chisato", hps)
with torch.no_grad():
    x_tst = stn_tst.to(torch.device('cpu')).unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(torch.device('cpu'))
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.6, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False, autoplay=True))

In [8]:
from scipy.io import wavfile
wavfile.write('./audio.wav', rate=hps.data.sampling_rate, data=audio)
print()

[ 0.00091306  0.00096463  0.00091717 ... -0.00024987 -0.00038512
 -0.00027993]


In [10]:
data = wavfile.read('./audio.wav')[1]
print(data)
data = data/32768
print(data)

[ 0.00091306  0.00096463  0.00091717 ... -0.00024987 -0.00038512
 -0.00027993]
[ 2.7864345e-08  2.9438313e-08  2.7989790e-08 ... -7.6254896e-09
 -1.1753064e-08 -8.5427523e-09]


## VCTK

In [ ]:
hps = utils.get_hparams_from_file("./configs/vctk_base.json")

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).cuda()
_ = net_g.eval()

_ = utils.load_checkpoint("/path/to/pretrained_vctk.pth", net_g, None)

In [ ]:
stn_tst = get_text("VITS is Awesome!", hps)
with torch.no_grad():
    x_tst = stn_tst.cuda().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
    sid = torch.LongTensor([4]).cuda()
    audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))

### Voice Conversion

In [ ]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))